In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import pandas as pd
import os
import cv2

def detect_objects_in_frames(frame_directory):
    """
    Detecta objetos en fotogramas de un directorio utilizando ResNet50.

    Args:
        frame_directory (str): Ruta al directorio que contiene los fotogramas.

    Returns:
        pandas.DataFrame: DataFrame con el número de fotograma, clase predicha y confianza.
    """
    model = ResNet50(weights='imagenet')
    results = []
    frame_files = sorted(os.listdir(frame_directory))

    for frame_number, frame_file in enumerate(frame_files):
        frame_path = os.path.join(frame_directory, frame_file)
        try:
            img = cv2.imread(frame_path)
            img = cv2.resize(img, (224, 224))
            img = image.img_to_array(img)
            img = preprocess_input(img)
            img = tf.expand_dims(img, axis=0)

            predictions = model.predict(img)
            decoded_predictions = decode_predictions(predictions, top=1)[0]
            predicted_class = decoded_predictions[0][1]
            confidence = float(decoded_predictions[0][2])

            results.append([frame_number, predicted_class, confidence])
        except Exception as e:
            print(f"Error processing {frame_file}: {e}")

    df = pd.DataFrame(results, columns=['frame_number', 'predicted_class', 'confidence'])
    return df

# Ruta a tus fotogramas
frame_directory = r"C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\frames"

# Ejecutar la detección de objetos y mostrar los resultados
df_results = detect_objects_in_frames(frame_directory)
print(df_results)